In [1]:
#getting and working with data
import pandas as pd
import numpy as np
import re
import os
import datetime as dt

#visualizing results
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
#import yellowbrick as yb

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import warnings; warnings.simplefilter('ignore')
np.set_printoptions(suppress=True)

In [3]:
#path for FIFI FOI request
path = 'C:/Users/Schindler/Documents/ProgrammingFun/FIFI/FIFI_data_through_5-12-19.xlsx'

In [67]:
#original file has multiple sheets (for different FIFI types)
whole_file = pd.ExcelFile(path)

print(whole_file.sheet_names)

combined_data = pd.DataFrame()
for sheet_name in whole_file.sheet_names:
    sheet_data = pd.DataFrame(data = pd.read_excel(whole_file, sheetname=sheet_name))
    print(sheet_name)
    print(sheet_data.columns.values)
    sheet_data['FIFI_category'] = [sheet_name]*sheet_data.shape[0]
    combined_data = pd.concat([combined_data, sheet_data], sort=False)

combined_data.reset_index(drop=True, inplace=True)
print(combined_data.shape)
combined_data.head()

['Abandoned Vehicle', 'Clogged Storm Drain', 'Community Walk', 'Dead Animal', 'Graffiti Report', 'Litter-Illegal Dumping-Needles', 'Overgrown Vegetation', 'Parking Enforcement', 'Pothole', 'Sign and Signal Maintenance', 'Streetlight Report', 'Other Inquiry']
Abandoned Vehicle
['Service Request Number' 'Created Date' 'Location' 'Location Details'
 'Description' 'License Number' 'State' 'Vehicle Make' 'Vehicle Color'
 'Length of Time Parked' 'Photo']
Clogged Storm Drain
['Service Request Number' 'Created Date' 'Location' 'Location Details'
 'Description' 'Location Details.1' 'Causing property damage?' 'Photo']
Community Walk
['Service Request Number' 'Created Date' 'Location' 'Location Details'
 'Description' 'Current Issue' 'Location Details.1' 'Photo']
Dead Animal
['Service Request Number' 'Created Date' 'Location' 'Location Details'
 'Description' 'Type of animal' 'On private property?' 'Photo']
Graffiti Report
['Service Request Number' 'Created Date' 'Location' 'Location Details'
 'D

,Service Request Number,Created Date,Location,Location Details,Description,License Number,State,Vehicle Make,Vehicle Color,Length of Time Parked,Photo,FIFI_category,Location Details.1,Causing property damage?,Current Issue,Type of animal,On private property?,What is the Graffiti on?,Location and Description,"Obscene, racial, or hateful?",Cross street or intersection?,Where?,Type of item?,Where is the vegetation growing,Vegetation Safety Issue,Parking Violation Concerning,License Plate Number,Location details,7 digit number on pole,Issue
0,13-00036779,2013-05-09 20:32:33,"5208 BEACON AVE S, SEATTLE, WA","5200 Beacon Ave S Seattle, WA 98108; XY: 12775...",Ford light blue,AHZ9829,NaN,Ford,Blue,Over 3 Days,NaN,Abandoned Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,13-00064098,2013-08-13 18:05:17,"2629 SW ANDOVER ST, SEATTLE, WA","26012799 SW Andover St, Seattle; XY: 1261975.9...",NaN,Acxo217,NaN,Suzuki,Other,Over 1 Week,http://servicerequest.seattle.gov/media/seattl...,Abandoned Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,13-00064109,2013-08-13 18:57:44,"402 BELMONT AVE E, SEATTLE, WA","327 Belmont Ave E, Seattle; XY: 1272796.132329...",I don't think this car has been moved in quite...,379 vuq,NaN,Volkswagen,Silver,Over 3 Days,http://servicerequest.seattle.gov/media/seattl...,Abandoned Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,13-00064128,2013-08-13 20:24:22,"5283 39TH AVE S, SEATTLE, WA","5239 39th Ave S Seattle, WA 98118; XY: 1282330...",Older Honda with flat tire,See photo,NaN,Honda,Blue,Over 1 month,http://servicerequest.seattle.gov/media/seattl...,Abandoned Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,13-00064164,2013-08-14 06:51:34,"4755 38TH AVE SW, SEATTLE, WA","XY: 1258575.3814046786, 207971.16513023284; La...",Green 4 door Volkswagen Golf,WA AGD9667,NaN,Volkswagen,Green,Over 1 month,http://servicerequest.seattle.gov/media/seattl...,Abandoned Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [87]:
#select data that includes a zip code
combined_data_withzip = combined_data[combined_data['Created Date'].dt.year > 2015]
print(combined_data_withzip.shape)
#create new column with zip code
combined_data_withzip['zip'] = combined_data_withzip.apply(lambda row: row['Location'].split()[-1], axis=1)
combined_data_withzip.head()

(204534, 30)


,Service Request Number,Created Date,Location,Location Details,Description,License Number,State,Vehicle Make,Vehicle Color,Length of Time Parked,Photo,FIFI_category,Location Details.1,Causing property damage?,Current Issue,Type of animal,On private property?,What is the Graffiti on?,Location and Description,"Obscene, racial, or hateful?",Cross street or intersection?,Where?,Type of item?,Where is the vegetation growing,Vegetation Safety Issue,Parking Violation Concerning,License Plate Number,Location details,7 digit number on pole,Issue,zip
5994,16-00000050,2016-01-01 10:17:27.000,"5900 34TH CT S, SEATTLE, WA 98118","5900 34TH CT S; XY: 1280854.342623405, 203954....","Gold car, appears to have a window that won't ...",065-XVB,WA,Mercedes-Benz,Other,Over 1 month,http://servicerequest.seattle.gov/media/seattl...,Abandoned Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98118
5995,16-00000051,2016-01-01 10:23:26.999,"5901 34TH CT S, SEATTLE, WA 98118","5901 34TH CT S; XY: 1280797.3824180984, 203903...",Dark grey z28 with tinted windows. Has been si...,AUK7257\nIn pic also,WA,Chevrolet,grey,Over 1 month,http://servicerequest.seattle.gov/media/seattl...,Abandoned Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98118
5996,16-00000074,2016-01-01 12:48:52.000,"806 20TH AVE, SEATTLE, WA 98122","806 20TH AVE; XY: 1277227.747773104, 225632.57...",Abandoned car over 3 weeks,ATJ1786,WA,Mercury,Silver,Over 1 month,http://servicerequest.seattle.gov/media/seattl...,Abandoned Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98122
5997,16-00000089,2016-01-01 14:11:37.000,"1704 MELROSE AVE, SEATTLE, WA 98122","1704 MELROSE AVE; XY: 1271749.9204300693, 2287...",Abandoned car has been here for over 4 weeks. ...,ASW5537,WA,Lincoln,Silver,Over 1 month,http://servicerequest.seattle.gov/media/seattl...,Abandoned Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98122
5998,16-00000092,2016-01-01 14:35:48.000,"6549 10TH AVE NW, SEATTLE, WA 98117","6549 10TH AVE NW; XY: 1261926.7043027526, 2509...",NaN,C71347C,WA,Ford,White,Over 1 month,http://servicerequest.seattle.gov/media/seattl...,Abandoned Vehicle,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,98117


In [96]:
zip_neighborhood_dic = {'98118', '98122', '98117', '98125', '98108', '98144', '98112',
       '98107', '98103', '98199', '98102', '98133', '98177', '98106',
       '98109', '98115', '98136', '98134', '98119', '98126', '98105',
       '98178', '98116', '98101', '98168', '98104', '98195', '98146',
       '98121', '98155', '98031', '98188', '98148', 
       '98166', '98057', '98164', '98028']

array(['98118', '98122', '98117', '98125', '98108', '98144', '98112',
       '98107', '98103', '98199', '98102', '98133', '98177', '98106',
       '98109', '98115', '98136', '98134', '98119', '98126', '98105',
       '98178', '98116', '98101', '98168', '98104', '98195', '98146',
       '98121', 'WA', '98155', '98031', '98188', '98148', 'ST', 'N',
       '98166', '98057', '98164', 'INVALID', 'S', 'SW', 'NE', '1299',
       '4448', '4408', '3501', '4609', 'AVE', '4533', '1467', '98028',
       '627', '4458', '1445'], dtype=object)

In [98]:
combined_data_withzip['zip'].value_counts() > 1000

98103       True
98107       True
98144       True
98118       True
98105       True
98108       True
98122       True
98115       True
98102       True
98109       True
98117       True
98125       True
98133       True
98104       True
98106       True
98112       True
98119       True
98199       True
98134       True
98126       True
98101       True
98116       True
98121       True
98136       True
98178       True
98177       True
98195      False
98146      False
98155      False
98168      False
98188      False
ST         False
WA         False
98166      False
N          False
98148      False
INVALID    False
S          False
NE         False
98031      False
98057      False
98164      False
4533       False
1299       False
3501       False
98028      False
SW         False
1467       False
4458       False
AVE        False
627        False
4408       False
1445       False
4609       False
4448       False
Name: zip, dtype: bool

In [ ]:
#-999 and 'None' are missing values so replace with 'Nan'
merge_data_first = merge_data_first.replace({-999.0: np.nan, 'None': np.nan})

In [ ]:
#explore missing data
print(merge_data_first.shape)
print(merge_data_first[merge_data_first['Group'] == 'T'].shape)
print(merge_data_first[merge_data_first['Group'] == 'T'].isna().sum().sort_values(ascending=False))
merge_data_first[merge_data_first['Group'] == 'T'].isna().sum().sort_values(ascending=False).plot(kind = 'hist')
plt.show()

In [ ]:
dep_vars = ['GType', 'Race', 'Hispanic', 'Handedness', 'ScreenAge',
       'Education', 'PsyEduc', 'cestotal', 'NSITot', 'TBITot',
       'CAPSTotal', 'PCLTot', 'PTSD_YN', 'PSQItot', 'PHQTot', 'BNITotIm',
       'BISTot', 'AUDITtot', 'AUDIT1', 'AUDIT2', 'AUDIT3', 'DA', 'DOPA',
       'NE', 'QKOIorA', 'QKOExpMil', 'QKOAllMil', 'QKOLife', 'QBlstExp',
       'QBEACRM', 'MnthSncBlst', 'QEDist_sum', 'QEDist_mean', 'DvpHA',
       'DvpHAAct', 'DvpHASlp', 'DvpHAMd', 'DvpHAStr', 'DvpBP', 'DvpBPAct',
       'DvpBPslp', 'DvpBPMd', 'DvpBPStr', 'tbiDizzy', 'tbiBalan',
       'tbiCoord', 'tbiHeada', 'tbiNaus', 'tbiVision', 'tbiLight',
       'tbiHear', 'tbiNoise', 'tbiTingl', 'tbiTstsml', 'tbiAppet',
       'tbiConc', 'tbiForget', 'tbiDecis', 'tbiSlow', 'tbiEnergy',
       'tbiSleep', 'tbiAnx', 'tbiSad', 'tbiIrrit', 'tbiOverw', 'tbiDisin',
       'tbiWithd', 'tbiRing', 'tbiMoods', 'tbiFight', 'tbiSpch', 'hrslp',
       'PSQI1hr', 'PSQI1min', 'PSQI2', 'PSQI3hr', 'PSQI3min', 'PSQI5a',
       'PSQI5b', 'PSQI5c', 'PSQI5d', 'PSQI5e', 'PSQI5f', 'PSQI5g',
       'PSQI5h', 'PSQI5i', 'PSQI5j', 'PSQI6', 'PSQI7', 'PSQI8', 'PSQI9',
       'PSQIc1', 'PSQIc2', 'PSQIc3', 'PSQIc4', 'PSQIc5', 'PSQIc6',
       'PSQIc7', 'PCL1', 'PCL2', 'PCL3', 'PCL4', 'PCL5', 'PCL6', 'PCL7',
       'PCL8', 'PCL9', 'PCL10', 'PCL11', 'PCL12', 'PCL13', 'PCL14',
       'PCL15', 'PCL16', 'PCL17', 'PHQ1', 'PHQ2', 'PHQ3', 'PHQ4', 'PHQ5',
       'PHQ6', 'PHQ7', 'PHQ8', 'PHQ9', 'BNI1Im', 'BNI2Im', 'BNI3Im',
       'BNI4Im', 'BNI5Im', 'BNI6Im', 'BNI7Im', 'BNI8Im', 'BNI9Im',
       'BNI10Im', 'BNI11', 'BISAtt', 'BISMtr', 'BISNonpl', 'PETType',
       'Frontal_Mid_l', 'Frontal_Mid_r', 'Insula_l', 'Insula_r',
       'Cingulum_Ant_l', 'Cingulum_Ant_r', 'Amygdala_l', 'Amygdala_r',
       'CaudateNucl_l', 'CaudateNucl_r', 'Putamen_l', 'Putamen_r',
       'Pallidum_l', 'Pallidum_r', 'Medulla', 'Midbrain', 'Pons']

In [ ]:
for param in dep_vars:
    print(param)
    plt.figure(figsize=(10,5))
    try:
        merge_data_first.groupby(['Group'])[param].mean().plot(kind='bar', yerr=merge_data_first.groupby(['Group'])[param].sem())
        plt.ylabel([param])
    #plt.savefig(str(param + '.png'))
        plt.show()
    except:
        pass

In [ ]:
corr = merge_data_first.groupby('Group').corr()
#fig, ax = plt.subplots(figsize=(40, 40))
#sns.heatmap(corr, center=0)

In [ ]:
AUDITC_cols = ['AUDITtot',
       'AUDIT1', 'AUDIT2', 'AUDIT3']

In [ ]:
merge_data_first = merge_data_first[merge_data_first["TBIID"] != 'C010']
merge_data_first = merge_data_first[merge_data_first["TBIID"] != 'T080']

In [ ]:
#corr.to_csv('corr.csv')
merge_data_first.to_csv('merge_data_first.csv')

In [ ]:
AUDIT_tot_counts = merge_data_first.groupby('Group')['AUDITtot'].value_counts()
AUDIT_1_counts = merge_data_first.groupby('Group')['AUDIT1'].value_counts()
AUDIT_2_counts = merge_data_first.groupby('Group')['AUDIT2'].value_counts()
AUDIT_3_counts = merge_data_first.groupby('Group')['AUDIT3'].value_counts()
AUDIT_tot_counts.to_csv('AUDIT_tot_counts.csv')
AUDIT_1_counts.to_csv('AUDIT_1_counts.csv')
AUDIT_2_counts.to_csv('AUDIT_2_counts.csv')
AUDIT_3_counts.to_csv('AUDIT_3_counts.csv')

In [ ]:
AUDIT_tot_perc = merge_data_first.groupby('Group')['AUDITtot'].value_counts() / merge_data_first.groupby('Group')['AUDITtot'].count()
AUDIT_1_perc = merge_data_first.groupby('Group')['AUDIT1'].value_counts() / merge_data_first.groupby('Group')['AUDIT1'].count()
AUDIT_2_perc = merge_data_first.groupby('Group')['AUDIT2'].value_counts() / merge_data_first.groupby('Group')['AUDIT2'].count()
AUDIT_3_perc = merge_data_first.groupby('Group')['AUDIT3'].value_counts() / merge_data_first.groupby('Group')['AUDIT3'].count()
AUDIT_tot_perc.to_csv('AUDIT_tot_perc.csv')
AUDIT_1_perc.to_csv('AUDIT_1_perc.csv')
AUDIT_2_perc.to_csv('AUDIT_2_perc.csv')
AUDIT_3_perc.to_csv('AUDIT_3_perc.csv')